# 1. loading datasets

In [1]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Get current CUDA device index (if available)
if torch.cuda.is_available():
    print("Current CUDA device index:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No CUDA devices found.")

PyTorch version: 2.4.1+cu118
CUDA available: True
Current CUDA device index: 0
CUDA device name: NVIDIA GeForce RTX 3070


# 2. NN model

We eill try to determine the optimal hyperparameters

In [ ]:
class MLPBuilder(nn.Module):

    def __init__(self, no_features, layers, no_labels):
        super().__init__()
        layer_list = []
        
        # Input layer
        layer_list.append(nn.Linear(no_features, layers[0]))
        layer_list.append(nn.ReLU())
        layer_list.append(nn.Dropout(p=0.2))

        # Hidden layers
        for i in range(len(layers) - 1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            layer_list.append(nn.ReLU())
            layer_list.append(nn.Dropout(p=0.2))

        # Output layer
        layer_list.append(nn.Linear(layers[-1], no_labels))
        layer_list.append(nn.Sigmoid())  # Assuming binary classification

        # Define the MLP stack as a sequential model
        self.mlp_stack = nn.Sequential(*layer_list)
    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits